In [37]:
import random
from itertools import combinations

# Lista de todos os locais
locais = ['Faminas', 'Bar do Lu', 'Bar da Fatinha', 'Praça do Rosário', 'Bar do Broa', 'Inove', 'Dule',
          'Mansão Gastropub', 'João Pinheiro', 'Bar Betinho', 'Hospital São Paulo', 'Armação', 'Cristo',
          'Rei do Cachorao', 'Santa Marcelina', 'Cachorao do Naldinho', 'Fest House', 'Lagoa da Gávea']

# Gerar subrotas aleatórias
def gerar_subrotas(locais, num_subrotas=100, tam_min=3, tam_max=15):
    subrotas = []
    for _ in range(num_subrotas):
        tam = random.randint(tam_min, tam_max)
        subrota = random.sample(locais, tam)
        subrotas.append(subrota)
    return subrotas

subrotas = gerar_subrotas(locais)

In [38]:
from sklearn.model_selection import train_test_split

# Dividir em treinamento, validação e teste
subrotas_treino, subrotas_temp = train_test_split(subrotas, test_size=0.3, random_state=42)
subrotas_validacao, subrotas_teste = train_test_split(subrotas_temp, test_size=0.5, random_state=42)
subrotas_validacao

[['Hospital São Paulo',
  'Rei do Cachorao',
  'Santa Marcelina',
  'Cachorao do Naldinho',
  'Inove',
  'Dule',
  'Faminas',
  'Praça do Rosário',
  'Bar da Fatinha',
  'João Pinheiro',
  'Bar Betinho',
  'Bar do Broa',
  'Bar do Lu',
  'Mansão Gastropub',
  'Cristo'],
 ['Mansão Gastropub',
  'Bar do Broa',
  'Cristo',
  'Santa Marcelina',
  'Bar Betinho',
  'Rei do Cachorao'],
 ['João Pinheiro',
  'Bar do Lu',
  'Hospital São Paulo',
  'Cristo',
  'Fest House',
  'Praça do Rosário',
  'Inove',
  'Faminas',
  'Bar da Fatinha',
  'Rei do Cachorao',
  'Cachorao do Naldinho',
  'Armação'],
 ['Inove', 'Rei do Cachorao', 'Cachorao do Naldinho'],
 ['Bar Betinho', 'Lagoa da Gávea', 'Santa Marcelina', 'Praça do Rosário'],
 ['Fest House',
  'João Pinheiro',
  'Mansão Gastropub',
  'Cachorao do Naldinho',
  'Lagoa da Gávea',
  'Bar do Broa',
  'Faminas',
  'Inove',
  'Dule',
  'Rei do Cachorao',
  'Santa Marcelina',
  'Hospital São Paulo'],
 ['Rei do Cachorao', 'Bar Betinho', 'Cristo'],
 ['Bar 

In [39]:
import pandas as pd

# Especificar o caminho do arquivo CSV
file_path = 'rotas.csv'

# Carregar o CSV, especificando que o índice está na primeira coluna
distances = pd.read_csv(file_path, index_col=0)

# Remover espaços em branco dos nomes das colunas e dos índices
distances.columns = distances.columns.str.strip()
distances.index = distances.index.str.strip()

# Verificar os dados carregados
print(distances.head())

# Converter o DataFrame para um dicionário de distâncias
dist_dict = distances.apply(pd.to_numeric, errors='coerce').to_dict()

# Remover valores NaN do dicionário
dist_dict = {k: {ik: iv for ik, iv in v.items() if not pd.isna(iv)} for k, v in dist_dict.items()}

# Exemplo de como acessar as distâncias
loc1 = 'Faminas'
loc2 = 'Bar do Lu'
distancia = dist_dict[loc1][loc2]
print(f"A distância entre {loc1} e {loc2} é {distancia}")


                  Faminas  Bar do Lu  Bar da fatinha  Praça do rosario  \
João pinheiro         4.6        2.2             5.5              0.60   
Faminas               0.0        4.3             3.5              4.80   
Mansão gastropub      3.5        1.9             3.0              2.40   
Bar da fatinha        3.5        2.6             0.0              4.30   
Bar do broa           5.2        3.0             5.8              0.85   

                  Bar do broa  Inove  Dule  Mansão gastropub  João pinheiro  \
João pinheiro            0.65    1.7  0.55               2.3            0.0   
Faminas                  5.20    3.7  4.30               4.1            4.6   
Mansão gastropub         2.90    1.4  1.70               0.0            2.3   
Bar da fatinha           4.70    3.3  4.30               3.0            4.2   
Bar do broa              0.00    2.4  1.00               2.9            0.5   

                  Bar betinho  Hospital São Paulo  Armação  Cristo  \
João pinhe

In [40]:
import random

# Lista de todos os locais
locais = list(distances.columns)

# Função para gerar subrotas aleatórias
def gerar_subrotas(locais, num_subrotas=100, tam_min=3, tam_max=7):
    subrotas = []
    for _ in range(num_subrotas):
        tam = random.randint(tam_min, tam_max)
        subrota = random.sample(locais, tam)
        subrotas.append(subrota)
    return subrotas

subrotas = gerar_subrotas(locais)


In [41]:
from sklearn.model_selection import train_test_split

# Dividir em treinamento, validação e teste
subrotas_treino, subrotas_temp = train_test_split(subrotas, test_size=0.3, random_state=42)
subrotas_validacao, subrotas_teste = train_test_split(subrotas_temp, test_size=0.5, random_state=42)


In [42]:
# Função de fitness adaptada para subrotas
def fitness_subrota(subrota):
    return sum(dist_dict[subrota[i]][subrota[i + 1]] for i in range(len(subrota) - 1))


In [43]:
# Operadores Genéticos
def generate_population(size, locations):
    population = []
    for _ in range(size):
        individual = locations[:]
        random.shuffle(individual)
        population.append(individual)
    return population

def select(population, fitnesses):
    total_fitness = sum(fitnesses)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual, fitness in zip(population, fitnesses):
        current += fitness
        if current > pick:
            return individual

def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]
    
    p2_iter = iter(parent2)
    for i in range(size):
        if not child[i]:
            while True:
                candidate = next(p2_iter)
                if candidate not in child:
                    child[i] = candidate
                    break
    return child

def mutate(individual):
    a, b = random.sample(range(len(individual)), 2)
    individual[a], individual[b] = individual[b], individual[a]

# Algoritmo de Programação Genética adaptado
def genetic_algorithm_subrotas(subrotas, population_size=100, generations=100):
    best_routes = []
    for subrota in subrotas:
        population = generate_population(population_size, subrota)
        for generation in range(generations):
            fitnesses = [fitness_subrota(ind) for ind in population]
            new_population = []
            for _ in range(population_size // 2):
                parent1 = select(population, fitnesses)
                parent2 = select(population, fitnesses)
                child1 = crossover(parent1, parent2)
                child2 = crossover(parent2, parent1)
                mutate(child1)
                mutate(child2)
                new_population.extend([child1, child2])
            population = new_population
        best_individual = min(population, key=fitness_subrota)
        best_routes.append((best_individual, fitness_subrota(best_individual)))
    return best_routes

# Exemplo de uso
best_routes_treino = genetic_algorithm_subrotas(subrotas_treino)

# Avaliação dos resultados
def avaliar_resultados(best_routes):
    distancias = [route[1] for route in best_routes]
    return sum(distancias) / len(distancias)

media_treino = avaliar_resultados(best_routes_treino)
print(f"Distância média no treinamento: {media_treino}")


Distância média no treinamento: 6.134571428571428


In [46]:
# Exemplo de uso
best_routes_validacao = genetic_algorithm_subrotas(subrotas_validacao)

media_validacao = avaliar_resultados(best_routes_validacao)
print(f"Distância média na validação: {media_validacao}")


KeyError: ('Inove', 'Santa marcelina')

In [44]:
# Função de fitness adaptada para subrotas
def fitness_subrota(subrota):
    return sum(distances[(subrota[i], subrota[i + 1])] for i in range(len(subrota) - 1))

def generate_population(population_size, subrota):
    return [random.sample(subrota, len(subrota)) for _ in range(population_size)]

def select(population, fitnesses):
    return random.choices(population, weights=[1 / (f + 1) for f in fitnesses])[0]

def crossover(parent1, parent2):
    child = parent1.copy()
    indices = sorted(random.sample(range(len(parent1)), 2))
    for i in range(indices[0], indices[1]):
        child[i] = parent2[i]
    return child

def mutate(child):
    indices = random.sample(range(len(child)), 2)
    child[indices[0]], child[indices[1]] = child[indices[1]], child[indices[0]]

# Algoritmo de Programação Genética adaptado
def genetic_algorithm_subrotas(subrotas, population_size=100, generations=100):
    best_routes = []
    for subrota in subrotas:
        population = generate_population(population_size, subrota)
        for generation in range(generations):
            fitnesses = [fitness_subrota(ind) for ind in population]
            new_population = []
            for _ in range(population_size // 2):
                parent1 = select(population, fitnesses)
                parent2 = select(population, fitnesses)
                child1 = crossover(parent1, parent2)
                child2 = crossover(parent2, parent1)
                mutate(child1)
                mutate(child2)
                new_population.extend([child1, child2])
            population = new_population
        best_individual = min(population, key=fitness_subrota)
        best_routes.append((best_individual, fitness_subrota(best_individual)))
    return best_routes

# Treinamento
best_routes_treino = genetic_algorithm_subrotas(subrotas_treino)

# Validação
best_routes_validacao = genetic_algorithm_subrotas(subrotas_validacao)

# Teste
best_routes_teste = genetic_algorithm_subrotas(subrotas_teste)

# Avaliação dos resultados
def avaliar_resultados(best_routes):
    distancias = [route[1] for route in best_routes]
    return sum(distancias) / len(distancias)

media_treino = avaliar_resultados(best_routes_treino)
media_validacao = avaliar_resultados(best_routes_validacao)
media_teste = avaliar_resultados(best_routes_teste)

print(f"Distância média no treinamento: {media_treino}")
print(f"Distância média na validação: {media_validacao}")
print(f"Distância média no teste: {media_teste}")


KeyError: ('Bar betinho', 'Praça do rosario')